<a href="https://colab.research.google.com/github/SharathTejavat/SharathTejavat/blob/main/real.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import pandas as pd
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

real = pd.read_csv('/content/drive/My Drive/Colab Notebooks/fer2013.csv')

# Load and preprocess the dataset (use FER-2013 or similar facial expression datasets)
def load_dataset():
    # Assuming FER-2013 dataset or similar format with "pixels" and "emotion" columns
    import pandas as pd

    data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/fer2013.csv')
    X = []
    y = []
    for i in range(len(data)):
        pixels = np.array(data['pixels'][i].split(), dtype='float32').reshape(48, 48)
        X.append(pixels)
        y.append(data['emotion'][i])
    X = np.array(X)
    y = np.array(y)
    X = X / 255.0  # Normalize pixel values
    y = to_categorical(y)  # One-hot encode emotion labels
    return X, y
real.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/Colab Notebooks/fer2013.csv'

In [ ]:

# Build the CNN model
def build_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(7, activation='softmax')  # Assuming 7 emotion classes
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Train and evaluate the model
def train_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = build_model((48, 48, 1))
    model.fit(X_train, y_train, epochs=15, batch_size=64, validation_data=(X_test, y_test))
    return model, X_test, y_test

# Real-time emotion detection with webcam
def emotion_detection(model):
    emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        detected_faces = faces.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in detected_faces:
            roi = gray[y:y+h, x:x+w]
            roi = cv2.resize(roi, (48, 48))
            roi = roi.reshape(1, 48, 48, 1) / 255.0
            prediction = model.predict(roi)
            emotion = emotion_labels[np.argmax(prediction)]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        cv2.imshow('Emotion Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Main function
if __name__ == "__main__":
    X, y = load_dataset()
    X = X.reshape(-1, 48, 48, 1)  # Reshape for CNN
    model, X_test, y_test = train_model(X, y)
    print("Model trained successfully. Starting real-time emotion detection.")
    emotion_detection(model)


In [ ]:
from IPython.display import display, Javascript

# JavaScript function to capture a photo from the webcam
def take_photo():
    js_code = """
        async function takePhoto() {
            const video = document.createElement('video');
            const stream = await navigator.mediaDevices.getUserMedia({ video: true });

            video.srcObject = stream;
            await new Promise((resolve) => video.onloadedmetadata = resolve);
            video.play();

            await new Promise((resolve) => setTimeout(resolve, 1000));

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getTracks().forEach(track => track.stop());

            return canvas.toDataURL('image/jpeg', 0.8);
        }
        google.colab.kernel.invokeFunction('notebook.capture_photo', [], {});
    """
    display(Javascript(js_code))

In [ ]:
from google.colab.output import eval_js
import numpy as np
import cv2
import base64

# Function to convert JavaScript image to OpenCV format
def js_to_image(js_reply):
    image_bytes = base64.b64decode(js_reply.split(',')[1])  # Decode base64 image
    nparr = np.frombuffer(image_bytes, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    return img

# Display JavaScript to capture image
take_photo()  # Ensure JavaScript function is available

# Wait and then capture the frame
photo_data = eval_js("takePhoto()")  # Trigger JavaScript function
captured_frame = js_to_image(photo_data)  # Convert image to OpenCV format

# Verify if frame is captured
if captured_frame is None:
    print("Error: Could not capture frame.")
else:
    print("Frame captured successfully!")

<IPython.core.display.Javascript object>

Frame captured successfully!


In [ ]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np

model=load_model('/content/drive/My Drive/Colab Notebooks/cnn_model.keras')
# Function to preprocess the captured frame for emotion detection
def preprocess_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    gray = cv2.resize(gray, (48, 48))  # Resize to 48x48 (input size of model)
    gray = gray.reshape(1, 48, 48, 1) / 255.0  # Normalize and reshape for model
    return gray

# Function to predict emotion from the frame using the trained model
def predict_emotion(frame, model):
    preprocessed_frame = preprocess_frame(frame)  # Preprocess the frame
    emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

    # Predict the emotion using the model
    prediction = model.predict(preprocessed_frame)
    emotion = emotion_labels[np.argmax(prediction)]  # Get the emotion with the highest probability

    return emotion

# Function to draw the emotion label on the frame (optional: face detection)
def draw_results(frame, emotion, x, y, w, h):
    cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)  # Draw rectangle around face
    cv2.putText(frame, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)  # Draw emotion label
    return frame

# Assuming captured_frame is the captured image from the webcam
captured_frame = js_to_image(photo_data)  # Convert the captured frame from JavaScript

# Detect faces in the captured frame (you may use OpenCV's face detection for this)
faces = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml').detectMultiScale(captured_frame, 1.3, 5)

# Process each detected face
for (x, y, w, h) in faces:
    face = captured_frame[y:y + h, x:x + w]  # Crop the face region

    # Predict the emotion for the detected face
    emotion = predict_emotion(face, model)

    # Draw the result on the captured frame (bounding box + emotion label)
    captured_frame = draw_results(captured_frame, emotion, x, y, w, h)

# Display the processed image with emotion label (in Colab)
from google.colab.patches import cv2_imshow
cv2_imshow(captured_frame)

# Optionally, save the processed image
cv2.imwrite("emotion_detected.jpg", captured_frame)

# Confirmation message
print("Emotion detected and image displayed successfully!")

ValueError: File not found: filepath=/content/drive/My Drive/Colab Notebooks/cnn_model.keras. Please ensure the file is an accessible `.keras` zip file.